# Problem Statement - Automatic Loan Approval/Rejection System using Explanable AI

## Implementation of Two steps approach: -

- Step 1 : Using the output from Case 1, if the loan is predicted to be approved, then use credit bureau data to predict their chances of default.

In [ ]:
from google.colab import drive
drive.mount('/content/drive', readonly=False)

In [ ]:
#!pip install xgboost
#!pip install imblearn

In [ ]:
#import cudf.pandas
#cudf.pandas.install()
import shap
import joblib
from sklearn.model_selection import GridSearchCV,StratifiedKFold
import xgboost
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sre_constants import MAX_REPEAT
import hashlib
import math
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score, classification_report
from imblearn.under_sampling import NearMiss, RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler, SMOTE
from sklearn.linear_model import LogisticRegression

## Save/Access the Processed Dataset

In [ ]:
#df_accepted.to_csv("processed.csv", index=False)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
dft = pd.read_csv("/content/drive/MyDrive/Datasets/processed.csv")
dft.head()

### Since there is Target Class Imbalance, we Resample the Dataset using NearMiss algorithms - https://imbalanced-learn.org/stable/references/generated/imblearn.under_sampling.NearMiss.html.

In [ ]:
def under_resample_dataset(X : pd.DataFrame, y : pd.DataFrame):
    #nm = NearMiss(n_jobs=-1)
    #X_res, y_res = nm.fit_resample(X, y)
    rus = RandomUnderSampler(random_state=42, sampling_strategy='majority', replacement=True)
    X_res, y_res = rus.fit_resample(X, y)
    return X_res, y_res

def over_resample_dataset(X : pd.DataFrame, y : pd.DataFrame):
    #ros = RandomOverSampler(sampling_strategy='minority', random_state=42)
    #X_res, y_res = ros.fit_resample(X, y)
    smote = SMOTE(sampling_strategy='minority', random_state=42, k_neighbors=10)
    X_res, y_res = smote.fit_resample(X, y)
    return X_res, y_res

In [ ]:
#random_forest_reg = RandomForestRegressor(n_estimators=29, random_state=42, verbose=True, n_jobs=-1)


y = dft['loan_status']
X = dft.drop('loan_status', axis=1)

#X, y = under_resample_dataset(X, y)
#X, y = over_resample_dataset(X, y)
#X, y = over_resample_dataset(X, y)
dft["loan_status"].value_counts()


## Stratified K-Fold Cross Validation

In [ ]:
def strat_kfold(X : pd.DataFrame, y : pd.DataFrame, model, folds = 5, sampling : bool = True):
    scores = list
    cv_score = {}
    acc_score = []
    prec_score = []
    rec_score = []
    f1_scr = []
    roc_auc_scr = []
    #confusion_matrix = []
    #classification_report = []

    validation = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)
    fold = 0
    for train_index, test_index in validation.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        if(sampling):
            #X_train, y_train = over_resample_dataset(X_train, y_train)
            X_train, y_train = over_resample_dataset(X_train, y_train)
            X_test, y_test = over_resample_dataset(X_test, y_test)

        model.fit(X_train, y_train)
        y_pred = None
        if(model.__class__.__name__ == 'RandomForestClassifier' or model.__class__.__name__ == "XGBClassifier"):
            y_prob = model.predict_proba(X_test)[:, 1]
            y_pred = (y_prob >= 0.3).astype(int)
        else:
            y_pred = model.predict(X_test)

        #y_pred = model.predict(X_test)

        print(f"Fold = {fold}")

        acc_score.append(accuracy_score(y_test, y_pred))
        prec_score.append(precision_score(y_test, y_pred))
        rec_score.append(recall_score(y_test, y_pred))
        f1_scr.append(f1_score(y_test, y_pred))
        roc_auc_scr.append(roc_auc_score(y_test, y_pred))
        #confusion_matrix.append(confusion_matrix(y_test, y_pred))
        #classification_report.append(classification_report(y_test, y_pred))
        #print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
        #print(f"Precision: {precision_score(y_test, y_pred):.4f}")
        #print(f"Recall: {recall_score(y_test, y_pred):.4f}")
        #print(f"F1 Score: {f1_score(y_test, y_pred):.4f}")
        #print(f"ROC AUC Score: {roc_auc_score(y_test, y_pred_proba):.4f}")

        print("\nConfusion Matrix:")
        print(confusion_matrix(y_test, y_pred))

        print("\nClassification Report:")
        print(classification_report(y_test, y_pred))

        fold += 1
    #cv_score['accuracy_score'] = np.mean(accuracy_score)
    #cv_score['precision_score'] = np.mean(precision_score)
    #cv_score['recall_score'] = np.mean(recall_score)
    #cv_score['f1_score'] = np.mean(f1_score)
    #cv_score['roc_auc_score'] = np.mean(roc_auc_score)

    print(f"Mean Accuracy: {np.mean(acc_score):.4f}")
    print(f"Mean Precision: {np.mean(prec_score):.4f}")
    print(f"Mean Recall: {np.mean(rec_score):.4f}")
    print(f"Mean F1 Score: {np.mean(f1_scr):.4f}")
    print(f"Mean ROC AUC Score: {np.mean(roc_auc_scr):.4f}")

    model = None
    return cv_score

In [ ]:
# Random Forest
#random_forest_classifier = RandomForestClassifier(n_estimators=8, criterion='gini', class_weight='balanced_subsample', n_jobs=-1, verbose=True)
#strat_kfold(X, y, model=random_forest_classifier, sampling=True, folds = 2)   # Un-comment this line to re-train the model

'''
# Logistic Regression
logistic_regression = LogisticRegression(solver='saga', random_state=42, n_jobs=-1, verbose=True)
strat_kfold(X, y, model=logistic_regression)

# XGBoost Classifier
xgboostclassifier = xgboost.XGBClassifier(n_estimators=8, sampling_method='uniform', random_state=42, n_jobs=-1, verbose=True)
strat_kfold(X, y, model=xgboostclassifier, folds=5)
'''

## Save and Export the Trained Model

In [ ]:
model_name = '/content/drive/MyDrive/Datasets/random_forest_classifier.sav'
#joblib.dump(random_forest_classifier, model_name)

## Simulate Credit Bureaue Data for the Customer
- Scenario 1
    * Customer is entirely new that is NO CREDIT BUREAUE DATA AVAILABLE for the customer.
        1. Flag the Customer as New (flag=0).
        2. Take Basic Financial details of the Customer.
        3. Simulate the Credit Bureaue data for the Customer.
        4. Assign a ID to the Customer and Append the details to the database/CSV.

- Scenario 2
    * Customer is an existing Customer and CREDIT BUREAUE DATA IS AVAILABLE for the customer.
        1. Flag the Customer as Existing (flag=1).
        2. Take the ID of the customer as input.
        3. Predict if the Customer will default based on the historical data as well as the Customer Own Data.

## Code to Simulate Credit Bureau Data for the Customer

In [ ]:
def generate_credit_bureau_features(basic_app_features):
    """
    Generate credit bureau features deterministically from ALL basic application features
    using mathematical transformations only
    """

    # Create comprehensive seed from ALL basic features
    seed_parts = []
    for key, value in basic_app_features.items():
        seed_parts.append(f"{key}{value}")
    seed_data = "".join(seed_parts)
    seed_hash = hashlib.md5(seed_data.encode()).hexdigest()
    seed_int = int(seed_hash[:8], 16)

    def deterministic_value(index, max_val, min_val=0, modifier=1.0):
        """Generate deterministic values using trigonometric functions"""
        angle = (seed_int + index * 137) % 360
        trig_val = abs(math.sin(math.radians(angle)) + math.cos(math.radians(angle * 3)))
        value = min_val + int((trig_val * (max_val - min_val) * modifier) % (max_val - min_val + 1))
        return max(min_val, min(max_val, value))

    # Extract ALL basic features with defaults
    loan_amnt = basic_app_features.get('loan_amnt', 10000)
    annual_inc = basic_app_features.get('annual_inc', 50000)
    emp_length = basic_app_features.get('emp_length', 3)
    application_type = basic_app_features.get('application_type', 'Individual')
    has_secondary_applicant = basic_app_features.get('has_secondary_applicant', 0)
    sec_app_fico_low = basic_app_features.get('sec_app_fico_range_low')
    sec_app_revol_util = basic_app_features.get('sec_app_revol_util')

    # Home ownership features
    home_ownership = 'RENT'  # Default
    for home_type in ['ANY', 'MORTGAGE', 'OWN', 'RENT', 'NONE', 'OTHER']:
        if basic_app_features.get(f'home_ownership_{home_type}', 0) == 1:
            home_ownership = home_type
            break

    # Loan purpose
    loan_purpose = 'debt_consolidation'  # Default
    for purpose in ['car', 'credit_card', 'debt_consolidation', 'home_improvement',
                   'house', 'major_purchase', 'medical', 'moving', 'other',
                   'renewable_energy', 'small_business', 'vacation', 'wedding', 'educational']:
        if basic_app_features.get(f'purpose_{purpose}', 0) == 1:
            loan_purpose = purpose
            break

    # Verification status
    verification_status = 'Not Verified'  # Default
    for status in ['Not Verified', 'Source Verified', 'Verified']:
        if basic_app_features.get(f'verification_status_{status.replace(" ", "_")}', 0) == 1:
            verification_status = status
            break

    # Loan term
    term_36_months = basic_app_features.get('term_36 months', 0)
    term_60_months = basic_app_features.get('term_60 months', 0)
    loan_term = 36 if term_36_months else 60

    # Calculate comprehensive factors from ALL basic features
    income_ratio = min(2.0, loan_amnt / max(1, annual_inc))
    employment_factor = min(1.0, emp_length / 10.0)

    # Home ownership factor (better credit for homeowners)
    home_owner_factor = 1.2 if home_ownership in ['MORTGAGE', 'OWN'] else 1.0

    # Loan purpose factor (riskier purposes get worse credit)
    purpose_risk_factors = {
        'debt_consolidation': 1.0, 'home_improvement': 1.1, 'house': 1.2,
        'car': 1.0, 'major_purchase': 1.0, 'medical': 0.9, 'credit_card': 0.9,
        'small_business': 0.8, 'educational': 1.1, 'wedding': 1.0,
        'moving': 0.9, 'vacation': 0.8, 'renewable_energy': 1.0, 'other': 0.9
    }
    purpose_factor = purpose_risk_factors.get(loan_purpose, 1.0)

    # Verification status factor
    verification_factors = {
        'Verified': 1.2,
        'Source Verified': 1.1,
        'Not Verified': 1.0
    }
    verification_factor = verification_factors.get(verification_status, 1.0)

    # Loan term factor (longer term = higher risk)
    term_factor = 0.9 if loan_term == 60 else 1.0

    # Application type factor
    app_type_factor = 1.1 if application_type == 'Joint' else 1.0

    # Combined risk modifier
    combined_modifier = (home_owner_factor * purpose_factor * verification_factor *
                        term_factor * app_type_factor)

    credit_features = {}

    # FICO & Credit Scores (300-850) - influenced by ALL basic factors
    base_score = 300 + deterministic_value(1, 550, modifier=combined_modifier)
    employment_bonus = min(50, emp_length * 5)
    income_bonus = min(30, int(annual_inc / 10000))
    # Additional bonuses based on home ownership and verification
    home_bonus = 20 if home_ownership in ['MORTGAGE', 'OWN'] else 0
    verification_bonus = 15 if verification_status == 'Verified' else 0
    credit_features['fico_score'] = min(850, base_score + employment_bonus +
                                       income_bonus + home_bonus + verification_bonus)

    # Delinquency History - influenced by employment and income
    emp_delinq_modifier = max(0.5, 1.0 - (emp_length * 0.05))
    income_delinq_modifier = max(0.5, 1.0 - (annual_inc / 200000))

    credit_features['delinq_2yrs'] = deterministic_value(2, 5, modifier=emp_delinq_modifier * income_delinq_modifier)
    credit_features['acc_now_delinq'] = deterministic_value(3, 3, modifier=emp_delinq_modifier)
    credit_features['pub_rec'] = deterministic_value(4, 4, modifier=emp_delinq_modifier)
    credit_features['tax_liens'] = deterministic_value(5, 2, modifier=income_delinq_modifier)

    # Payment delinquency metrics - influenced by loan purpose
    purpose_delinq_modifier = 1.2 if loan_purpose in ['small_business', 'medical'] else 1.0
    credit_features['num_tl_30dpd'] = deterministic_value(6, 8, modifier=purpose_delinq_modifier)
    credit_features['num_tl_120dpd_2m'] = deterministic_value(7, 3, modifier=purpose_delinq_modifier)
    credit_features['num_tl_90g_dpd_24m'] = deterministic_value(8, 5, modifier=purpose_delinq_modifier)
    credit_features['num_accts_ever_120_pd'] = deterministic_value(9, 6, modifier=purpose_delinq_modifier)

    # Credit Accounts - influenced by income and employment
    income_account_modifier = min(2.0, annual_inc / 50000)
    base_accounts = max(3, int(emp_length * 2))
    total_accounts = base_accounts + deterministic_value(10, 40, modifier=income_account_modifier)
    credit_features['open_acc'] = total_accounts

    # Account type distribution influenced by home ownership and loan purpose
    home_installment_modifier = 1.5 if home_ownership in ['MORTGAGE', 'OWN'] else 1.0
    installment_accounts = deterministic_value(11, min(20, total_accounts), modifier=home_installment_modifier)
    credit_features['open_act_il'] = installment_accounts
    credit_features['num_il_tl'] = installment_accounts

    # Revolving accounts influenced by loan purpose
    purpose_rev_modifier = 1.3 if loan_purpose == 'credit_card' else 1.0
    rev_accounts = deterministic_value(12, min(25, total_accounts - installment_accounts), modifier=purpose_rev_modifier)
    credit_features['num_rev_accts'] = rev_accounts
    credit_features['num_op_rev_tl'] = rev_accounts

    # Bankcard accounts
    bankcard_accounts = deterministic_value(13, min(15, rev_accounts))
    credit_features['num_bc_tl'] = bankcard_accounts
    credit_features['num_rev_tl_bal_gt_0'] = deterministic_value(14, rev_accounts)

    # Credit Utilization & Balances - heavily influenced by income
    income_limit_modifier = min(3.0, annual_inc / 50000)
    total_credit_limit = max(5000, annual_inc * 0.6 * income_limit_modifier)
    credit_features['tot_hi_cred_lim'] = total_credit_limit + deterministic_value(15, 50000, modifier=income_limit_modifier)

    # Revolving calculations influenced by loan amount
    loan_util_modifier = min(1.5, loan_amnt / 10000)
    rev_util_rate = deterministic_value(16, 100, modifier=loan_util_modifier) / 100.0
    credit_features['revol_util'] = rev_util_rate * 100
    credit_features['revol_bal'] = int(total_credit_limit * rev_util_rate * 0.7)

    credit_features['tot_cur_bal'] = credit_features['revol_bal'] + deterministic_value(17, 50000, modifier=income_limit_modifier)
    credit_features['max_bal_bc'] = deterministic_value(18, 20000, modifier=income_limit_modifier)

    # Utilization metrics influenced by verification status
    verification_util_modifier = 0.8 if verification_status == 'Verified' else 1.0
    credit_features['all_util'] = deterministic_value(19, 85, modifier=verification_util_modifier)
    credit_features['bc_util'] = deterministic_value(20, 90, modifier=verification_util_modifier)
    credit_features['bc_open_to_buy'] = max(0, total_credit_limit - credit_features['revol_bal'])

    # Recent Activity - influenced by loan term and application type
    term_activity_modifier = 1.2 if loan_term == 60 else 1.0
    joint_app_modifier = 1.3 if application_type == 'Joint' else 1.0

    credit_features['open_acc_6m'] = deterministic_value(21, 8, modifier=term_activity_modifier)
    credit_features['open_il_12m'] = deterministic_value(22, 5, modifier=joint_app_modifier)
    credit_features['open_il_24m'] = credit_features['open_il_12m'] + deterministic_value(23, 3, modifier=joint_app_modifier)
    credit_features['open_rv_12m'] = deterministic_value(24, 6, modifier=term_activity_modifier)
    credit_features['open_rv_24m'] = credit_features['open_rv_12m'] + deterministic_value(25, 4, modifier=term_activity_modifier)
    credit_features['num_tl_op_past_12m'] = credit_features['open_il_12m'] + credit_features['open_rv_12m']
    credit_features['inq_fi'] = deterministic_value(26, 12, modifier=term_activity_modifier)
    credit_features['acc_open_past_24mths'] = credit_features['open_il_24m'] + credit_features['open_rv_24m']

    # Time-Based Metrics - influenced by employment length
    emp_time_modifier = min(2.0, emp_length / 3.0)
    credit_features['mths_since_rcnt_il'] = deterministic_value(27, 36, modifier=1/emp_time_modifier)
    credit_features['mo_sin_old_il_acct'] = max(12, deterministic_value(28, 240, modifier=emp_time_modifier))
    credit_features['mo_sin_old_rev_tl_op'] = max(6, deterministic_value(29, 180, modifier=emp_time_modifier))
    credit_features['mo_sin_rcnt_rev_tl_op'] = deterministic_value(30, 24, modifier=1/emp_time_modifier)
    credit_features['mo_sin_rcnt_tl'] = deterministic_value(31, 18, modifier=1/emp_time_modifier)
    credit_features['mths_since_recent_bc'] = deterministic_value(32, 24, modifier=1/emp_time_modifier)
    credit_features['mths_since_recent_inq'] = deterministic_value(33, 12, modifier=1/emp_time_modifier)

    # Delinquency time metrics - influenced by home ownership
    home_delinq_modifier = 0.7 if home_ownership in ['MORTGAGE', 'OWN'] else 1.0
    credit_features['mths_since_recent_bc_dlq'] = deterministic_value(34, 60, modifier=home_delinq_modifier)
    if credit_features['mths_since_recent_bc_dlq'] == 0:
        credit_features['mths_since_recent_bc_dlq'] = 999

    credit_features['mths_since_recent_revol_delinq'] = deterministic_value(35, 48, modifier=home_delinq_modifier)
    if credit_features['mths_since_recent_revol_delinq'] == 0:
        credit_features['mths_since_recent_revol_delinq'] = 999

    # Additional Credit Metrics
    credit_features['il_util'] = deterministic_value(36, 80)

    # Mortgage accounts directly influenced by home ownership
    if home_ownership == 'MORTGAGE':
        credit_features['mort_acc'] = 1 + deterministic_value(37, 3)
    else:
        credit_features['mort_acc'] = deterministic_value(37, 2)

    credit_features['num_actv_bc_tl'] = max(0, bankcard_accounts - deterministic_value(38, bankcard_accounts))
    credit_features['num_actv_rev_tl'] = max(0, rev_accounts - deterministic_value(39, rev_accounts))

    credit_features['num_bc_sats'] = deterministic_value(40, 8)
    credit_features['percent_bc_gt_75'] = deterministic_value(41, 100)

    # Percentage never delinquent - influenced by verification status
    verification_delinq_modifier = 1.2 if verification_status == 'Verified' else 1.0
    never_delinq_pct = max(50, 100 - (credit_features['delinq_2yrs'] * 10 +
                                     credit_features['num_tl_30dpd'] * 5))
    credit_features['pct_tl_nvr_dlq'] = min(100, never_delinq_pct * verification_delinq_modifier)

    # Balance metrics
    if total_accounts > 0:
        credit_features['avg_cur_bal'] = credit_features['tot_cur_bal'] / total_accounts
    else:
        credit_features['avg_cur_bal'] = 0

    # Collection amount influenced by loan purpose
    purpose_coll_modifier = 1.5 if loan_purpose in ['medical', 'small_business'] else 1.0
    credit_features['tot_coll_amt'] = deterministic_value(42, 10000, modifier=purpose_coll_modifier)

    # Debt settlement influenced by income and employment
    # debt_settlement_chance = max(1, min(10, 10 - emp_length + int(income_ratio * 5)))
    # credit_features['debt_settlement_flag'] = 1 if deterministic_value(43, 100) < debt_settlement_chance else 0

    # Secondary applicant worth (if applicable)
    # if has_secondary_applicant:
    #     sec_app_modifier = deterministic_value(44, 100) / 100.0
    #     # Higher worth for verified joint applications
    #     if verification_status == 'Verified':
    #         sec_app_modifier *= 1.5
    #     credit_features['sec_app_worth'] = annual_inc * sec_app_modifier
    # else:
    #     credit_features['sec_app_worth'] = 0


    # DTI - influenced by income and loan purpose
    base_dti = np.where(
        loan_purpose == 'debt_consolidation', 25,
        np.where(loan_purpose == 'credit_card', 22,
                np.where(loan_amnt / annual_inc > 0.3, 28, 18))
    )
    credit_features['dti'] = np.clip(base_dti + deterministic_value(43, 10, min_val=-5), 5, 45)

    return credit_features

# Example usage with ALL basic features:
basic_features = {
    # Loan Details
    'loan_amnt': 15000,
    'term_36 months': 1,
    'term_60 months': 0,

    # Personal & Financial Information
    'emp_length': 5,
    'annual_inc': 75000,
    'application_type': 'Individual',
    'has_secondary_applicant': 1,
    'sec_app_fico_range_low': 550,
    'sec_app_revol_util': 21,

    # Credit

    # Home Ownership (one-hot encoded)
    'home_ownership_MORTGAGE': 1,
    'home_ownership_ANY': 0,
    'home_ownership_OWN': 0,
    'home_ownership_RENT': 0,
    'home_ownership_NONE': 0,
    'home_ownership_OTHER': 0,

    # Loan Purpose (one-hot encoded)
    'purpose_debt_consolidation': 1,
    'purpose_car': 0,
    'purpose_credit_card': 0,
    'purpose_home_improvement': 0,
    'purpose_house': 0,
    'purpose_major_purchase': 0,
    'purpose_medical': 0,
    'purpose_moving': 0,
    'purpose_other': 0,
    'purpose_renewable_energy': 0,
    'purpose_small_business': 0,
    'purpose_vacation': 0,
    'purpose_wedding': 0,
    'purpose_educational': 0,

    # Verification Status (one-hot encoded)
    'verification_status_Verified': 1,
    'verification_status_Not_Verified': 0,
    'verification_status_Source_Verified': 0
}

#credit_bureau_data = generate_credit_bureau_features(basic_features)

# Print results
#print("Generated Credit Bureau Features:")
#for feature, value in credit_bureau_data.items():
#    print(f"{feature}: {value}")

## Function to process the Incoming Customer Data

In [ ]:
import json
def incoming_data_process(data_str):
    #data_str = data.replace("'", '"')
    data_dict = json.loads(data_str)
    #print(f"Loaded Json data = {data_dict}")
    dict_customer = {}

    customer_type = data_dict["customer_type"]
    loan_amt = data_dict['loan_amnt']
    term = data_dict['term']
    emp_length = data_dict['emp_length']
    annual_inc = data_dict['annual_inc']
    application_type = data_dict['application_type']
    has_secondary_applicant = data_dict['has_secondary_applicant']
    home_ownership = data_dict['home_ownership']
    loan_purpose = data_dict['loan_purpose']
    verification_status = data_dict['verification_status']

    if(term == '36 months'):
        term_36_months = 1
        term_60_months = 0
    else:
        term_36_months = 0
        term_60_months = 1

    if(application_type == 'Individual'):
        application_type = 0
    else:
        application_type = 1

    if(has_secondary_applicant == 'Yes'):
        has_secondary_applicant = 1

        if((int(data_dict['sec_app_fico_range_low']) >= 500) and (int(data_dict['sec_app_revol_util'] < 30))):
            sec_app_worth = 3
        elif((int(data_dict['sec_app_fico_range_low']) >= 500) and (int(data_dict['sec_app_revol_util'] >= 30))):
            sec_app_worth = 2
        elif((int(data_dict['sec_app_fico_range_low']) < 500) and (int(data_dict['sec_app_revol_util'] >= 30))):
            sec_app_worth = 1
        elif((int(data_dict['sec_app_fico_range_low']) < 500) and (int(data_dict['sec_app_revol_util'] < 30))):
            sec_app_worth = 0
    else:
        has_secondary_applicant = 0
        sec_app_worth = 0

    if(home_ownership == 'MORTGAGE'):
        home_ownership_MORTGAGE = 1
        home_ownership_ANY = 0
        home_ownership_OWN = 0
        home_ownership_RENT = 0
        home_ownership_NONE = 0
        home_ownership_OTHER = 0
    elif(home_ownership == 'ANY'):
        home_ownership_MORTGAGE = 0
        home_ownership_ANY = 1
        home_ownership_OWN = 0
        home_ownership_RENT = 0
        home_ownership_NONE = 0
        home_ownership_OTHER = 0
    elif(home_ownership == 'OWN'):
        home_ownership_MORTGAGE = 0
        home_ownership_ANY = 0
        home_ownership_OWN = 1
        home_ownership_RENT = 0
        home_ownership_NONE = 0
        home_ownership_OTHER = 0
    elif(home_ownership == 'RENT'):
        home_ownership_MORTGAGE = 0
        home_ownership_ANY = 0
        home_ownership_OWN = 0
        home_ownership_RENT = 1
        home_ownership_NONE = 0
        home_ownership_OTHER = 0
    elif(home_ownership == 'NONE'):
        home_ownership_MORTGAGE = 0
        home_ownership_ANY = 0
        home_ownership_OWN = 0
        home_ownership_RENT = 0
        home_ownership_NONE = 1
        home_ownership_OTHER = 0
    elif(home_ownership == 'OTHER'):
        home_ownership_MORTGAGE = 0
        home_ownership_ANY = 0
        home_ownership_OWN = 0
        home_ownership_RENT = 0
        home_ownership_NONE = 0
        home_ownership_OTHER = 1

    if(loan_purpose == 'debt_consolidation'):
        purpose_debt_consolidation = 1
        purpose_car = 0
        purpose_credit_card = 0
        purpose_home_improvement = 0
        purpose_house = 0
        purpose_major_purchase = 0
        purpose_medical = 0
        purpose_moving = 0
        purpose_other = 0
        purpose_renewable_energy = 0
        purpose_small_business = 0
        purpose_vacation = 0
        purpose_wedding = 0
        purpose_educational = 0
    elif(loan_purpose == 'car'):
        purpose_debt_consolidation = 0
        purpose_car = 1
        purpose_credit_card = 0
        purpose_home_improvement = 0
        purpose_house = 0
        purpose_major_purchase = 0
        purpose_medical = 0
        purpose_moving = 0
        purpose_other = 0
        purpose_renewable_energy = 0
        purpose_small_business = 0
        purpose_vacation = 0
        purpose_wedding = 0
        purpose_educational = 0
    elif(loan_purpose == 'credit_card'):
        purpose_debt_consolidation = 0
        purpose_car = 0
        purpose_credit_card = 1
        purpose_home_improvement = 0
        purpose_house = 0
        purpose_major_purchase = 0
        purpose_medical = 0
        purpose_moving = 0
        purpose_other = 0
        purpose_renewable_energy = 0
        purpose_small_business = 0
        purpose_vacation = 0
        purpose_wedding = 0
        purpose_educational = 0
    elif(loan_purpose == 'home_improvement'):
        purpose_debt_consolidation = 0
        purpose_car = 0
        purpose_credit_card = 0
        purpose_home_improvement = 1
        purpose_house = 0
        purpose_major_purchase = 0
        purpose_medical = 0
        purpose_moving = 0
        purpose_other = 0
        purpose_renewable_energy = 0
        purpose_small_business = 0
        purpose_vacation = 0
        purpose_wedding = 0
        purpose_educational = 0
    elif(loan_purpose == 'house'):
        purpose_debt_consolidation = 0
        purpose_car = 0
        purpose_credit_card = 0
        purpose_home_improvement = 0
        purpose_house = 1
        purpose_major_purchase = 0
        purpose_medical = 0
        purpose_moving = 0
        purpose_other = 0
        purpose_renewable_energy = 0
        purpose_small_business = 0
        purpose_vacation = 0
        purpose_wedding = 0
        purpose_educational = 0
    elif(loan_purpose == 'major_purchase'):
        purpose_debt_consolidation = 0
        purpose_car = 0
        purpose_credit_card = 0
        purpose_home_improvement = 0
        purpose_house = 0
        purpose_major_purchase = 1
        purpose_medical = 0
        purpose_moving = 0
        purpose_other = 0
        purpose_renewable_energy = 0
        purpose_small_business = 0
        purpose_vacation = 0
        purpose_wedding = 0
        purpose_educational = 0
    elif(loan_purpose == 'medical'):
        purpose_debt_consolidation = 0
        purpose_car = 0
        purpose_credit_card = 0
        purpose_home_improvement = 0
        purpose_house = 0
        purpose_major_purchase = 0
        purpose_medical = 1
        purpose_moving = 0
        purpose_other = 0
        purpose_renewable_energy = 0
        purpose_small_business = 0
        purpose_vacation = 0
        purpose_wedding = 0
        purpose_educational = 0
    elif(loan_purpose == 'moving'):
        purpose_debt_consolidation = 0
        purpose_car = 0
        purpose_credit_card = 0
        purpose_home_improvement = 0
        purpose_house = 0
        purpose_major_purchase = 0
        purpose_medical = 0
        purpose_moving = 1
        purpose_other = 0
        purpose_renewable_energy
        purpose_small_business = 0
        purpose_vacation = 0
        purpose_wedding = 0
        purpose_educational = 0
    elif(loan_purpose == 'other'):
        purpose_debt_consolidation = 0
        purpose_car = 0
        purpose_credit_card = 0
        purpose_home_improvement = 0
        purpose_house = 0
        purpose_major_purchase = 0
        purpose_medical = 0
        purpose_moving = 0
        purpose_other = 1
        purpose_renewable_energy
        purpose_small_business = 0
        purpose_vacation = 0
        purpose_wedding = 0
        purpose_educational = 0
    elif(loan_purpose == 'renewable_energy'):
        purpose_debt_consolidation = 0
        purpose_car = 0
        purpose_credit_card = 0
        purpose_home_improvement = 0
        purpose_house = 0
        purpose_major_purchase = 0
        purpose_medical = 0
        purpose_moving = 0
        purpose_other = 0
        purpose_renewable_energy = 1
        purpose_small_business = 0
        purpose_vacation = 0
        purpose_wedding = 0
        purpose_educational = 0
    elif(loan_purpose == 'small_business'):
        purpose_debt_consolidation = 0
        purpose_car = 0
        purpose_credit_card = 0
        purpose_home_improvement = 0
        purpose_house = 0
        purpose_major_purchase = 0
        purpose_medical = 0
        purpose_moving = 0
        purpose_other = 0
        purpose_renewable_energy = 0
        purpose_small_business = 1
        purpose_vacation = 0
        purpose_wedding = 0
        purpose_educational = 0
    elif(loan_purpose == 'vacation'):
        purpose_debt_consolidation = 0
        purpose_car = 0
        purpose_credit_card = 0
        purpose_home_improvement = 0
        purpose_house = 0
        purpose_major_purchase = 0
        purpose_medical = 0
        purpose_moving = 0
        purpose_other = 0
        purpose_renewable_energy = 0
        purpose_small_business = 0
        purpose_vacation = 1
        purpose_wedding = 0
        purpose_educational = 0
    elif(loan_purpose == "wedding"):
        purpose_debt_consolidation = 0
        purpose_car = 0
        purpose_credit_card = 0
        purpose_home_improvement = 0
        purpose_house = 0
        purpose_major_purchase = 0
        purpose_medical = 0
        purpose_moving = 0
        purpose_other = 0
        purpose_renewable_energy = 0
        purpose_small_business = 0
        purpose_vacation = 0
        purpose_wedding = 1
        purpose_educational = 0
    elif(loan_purpose == "educational"):
        purpose_debt_consolidation = 0
        purpose_car = 0
        purpose_credit_card = 0
        purpose_home_improvement = 0
        purpose_house = 0
        purpose_major_purchase = 0
        purpose_medical = 0
        purpose_moving = 0
        purpose_other = 0
        purpose_renewable_energy = 0
        purpose_small_business = 0
        purpose_vacation = 0
        purpose_wedding = 0
        purpose_educational = 1
    else:
        purpose_debt_consolidation = 0
        purpose_car = 0
        purpose_credit_card = 0
        purpose_home_improvement = 0
        purpose_house = 0
        purpose_major_purchase = 0
        purpose_medical = 0
        purpose_moving = 0
        purpose_other = 1
        purpose_renewable_energy
        purpose_small_business = 0
        purpose_vacation = 0
        purpose_wedding = 0
        purpose_educational = 0

    if(verification_status == 'Verified'):
        verification_status_Verified = 1
        verification_status_Not_Verified = 0
        verification_status_Source_Verified = 0
    elif(verification_status == 'Not Verified'):
        verification_status_Verified = 0
        verification_status_Not_Verified = 1
        verification_status_Source_Verified = 0
    elif(verification_status == 'Source Verified'):
        verification_status_Verified = 0
        verification_status_Not_Verified = 0
        verification_status_Source_Verified = 1

    dict_customer = {
        'loan_amnt': loan_amt,
        'term_36 months': term_36_months,
        'term_60 months': term_60_months,
        'emp_length': emp_length,
        'annual_inc': annual_inc,
        'application_type': application_type,
        'has_secondary_applicant': has_secondary_applicant,
        'home_ownership_MORTGAGE': home_ownership_MORTGAGE,
        'home_ownership_ANY': home_ownership_ANY,
        'home_ownership_OWN': home_ownership_OWN,
        'home_ownership_RENT': home_ownership_RENT,
        'home_ownership_NONE': home_ownership_NONE,
        'home_ownership_OTHER': home_ownership_OTHER,
        'purpose_debt_consolidation': purpose_debt_consolidation,
        'purpose_car': purpose_car,
        'purpose_credit_card': purpose_credit_card,
        'purpose_home_improvement': purpose_home_improvement,
        'purpose_house': purpose_house,
        'purpose_major_purchase': purpose_major_purchase,
        'purpose_medical': purpose_medical,
        'purpose_moving': purpose_moving,
        'purpose_other': purpose_other,
        'purpose_renewable_energy': purpose_renewable_energy,
        'purpose_small_business': purpose_small_business,
        'purpose_vacation': purpose_vacation,
        'purpose_wedding': purpose_wedding,
        'purpose_educational': purpose_educational,
        'verification_status_Verified': verification_status_Verified,
        'verification_status_Not Verified': verification_status_Not_Verified,
        'verification_status_Source Verified': verification_status_Source_Verified,
        'sec_app_worth': sec_app_worth
        #'sec_app_fico_low_range' : sec_app_fico_range_low,
        #'sec_app_revol_util' : sec_app_revol_util
        #

    }

    #df_customer = pd.DataFrame(dict_customer, index=[0])

    # Home ownership
    # Customer Type = 0 -> New Customer
    if(customer_type == 0):
        pass
    # Customer Type = 1 -> Existing Customer
    else:
        pass


    return dict_customer

## Incoming Data Pre-processing Pipeline

In [ ]:
def customer_data_preprocessing(data : str):
    print("Incoming Data")
    customer_data = incoming_data_process(data)
    print("Step 1 done")
    credit_bureau_data = generate_credit_bureau_features(customer_data)
    print("Step 2 done")
    X_columns = ['loan_amnt', 'emp_length', 'annual_inc', 'dti', 'delinq_2yrs', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'application_type', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'inq_fi', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_bc_dlq', 'mths_since_recent_inq', 'mths_since_recent_revol_delinq', 'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'pct_tl_nvr_dlq', 'percent_bc_gt_75', 'tax_liens', 'tot_hi_cred_lim', 'term_36 months', 'term_60 months', 'home_ownership_ANY', 'home_ownership_MORTGAGE', 'home_ownership_OWN', 'home_ownership_RENT', 'purpose_car', 'purpose_credit_card', 'purpose_debt_consolidation', 'purpose_home_improvement', 'purpose_house', 'purpose_major_purchase', 'purpose_medical', 'purpose_moving', 'purpose_other', 'purpose_renewable_energy', 'purpose_small_business', 'purpose_vacation', 'verification_status_Not Verified', 'verification_status_Source Verified', 'verification_status_Verified', 'has_secondary_applicant', 'sec_app_worth', 'fico_score', 'purpose_wedding', 'purpose_educational', 'home_ownership_NONE', 'home_ownership_OTHER']
    df_basic = pd.DataFrame(customer_data, index=[0])
    df_credit_bureau = pd.DataFrame(credit_bureau_data, index=[0])
    print("Step 3 done")
    df_simulated_data = pd.concat([df_basic, df_credit_bureau], axis=1)
    df_simulated_data = df_simulated_data.reindex(columns=X_columns)
    print("Step 4 done")
    return df_simulated_data

## Test Data

In [ ]:
loan_amt = 25000
term = '36 months'
emp_length = 3
annual_inc = 500000
application_type = 'Individual'
has_secondary_applicant = 'No'
sec_app_fico_range_low = 0
sec_app_revol_util = 0
home_ownership = 'RENT'
loan_purpose = 'wedding'
verification_status = 'Not Verified'

data = {
    'customer_type' : 0,
    'loan_amnt': loan_amt,
    'term': term,
    'emp_length': emp_length,
    'annual_inc': annual_inc,
    'application_type': application_type,
    'has_secondary_applicant': has_secondary_applicant,
    'sec_app_fico_range_low': sec_app_fico_range_low,
    'sec_app_revol_util': sec_app_revol_util,
    'home_ownership': home_ownership,
    'loan_purpose': loan_purpose,
    'verification_status': verification_status
}

#print(f"Jsoned Data = {json.dumps(data)}")
df_simulated_data = customer_data_preprocessing(json.dumps(data))

#print(f"Customer Data = {customer_data}")

In [ ]:
random_forest_classifier = joblib.load('/content/drive/MyDrive/Datasets/random_forest_classifier.sav')
y_pred = random_forest_classifier.predict(df_simulated_data)
y_pred

## SHAP Explainer

In [ ]:
def shap_explainer(df_simulated_data, model):
    explainer = shap.TreeExplainer(model)
    shap_values_cust = explainer.shap_values(df_simulated_data)
    shap_values_cust_0 = shap_values_cust[:, :, 0]

    #print(explainer.expected_value)
    #print(shap_values_cust_0[0])

    # Uncomment to see Waterfall model of Features importance and their contribution to the Random Forest Prediction
    '''
    shap.plots._waterfall.waterfall_legacy(
        explainer.expected_value[0],    # expected value for class 0
        shap_values_cust_0[0],         # SHAP values for that row
        feature_names=df_simulated_data.columns
    )
    '''
    return shap_values_cust_0

shap_values = shap_explainer(df_simulated_data, random_forest_classifier)

## SHAP Explainers

In [ ]:
def get_top_features_df(shap_values, feature_names, top_n=5):
    """
    Return top features as a DataFrame
    """
    if isinstance(shap_values, list):
        shap_array = shap_values[0][0]  # For binary classification
    else:
        shap_array = shap_values[0]     # For regression or newer API

    # Create DataFrame
    shap_df = pd.DataFrame({
        'feature': feature_names,
        'shap_value': shap_array,
        'abs_shap': np.abs(shap_array)
    })

    # Sort by absolute value and get top N
    top_df = shap_df.sort_values('abs_shap', ascending=False).head(top_n)

    return top_df

top_features = get_top_features_df(shap_values, df_simulated_data.columns, top_n=5)
print(top_features)

## SHAP Explainers to Text Description


In [ ]:
def generate_shap_descriptions(df_shap_features : pd.DataFrame, model_prediction : int):
    df_loan_columns = pd.read_csv("/content/drive/MyDrive/Datasets/loan_columns.csv")
    #print(df_loan_columns.head())
    print(df_loan_columns[df_loan_columns['Column Name'] == 'term_36 months']["Full Form"].squeeze())
    # Default Case
    if(model_prediction == 0):
        print("We regret to inform you that your Loan Application is rejected. Following are the possible reasons we found from your Data\n")
        desc = ""
        for _, row in df_shap_features.iterrows():
            feature_name = row['feature']
            #print(f"feature = {feature_name}")
            shap_value = row['shap_value']
            if(shap_value < 0):
                desc += f"{df_loan_columns[df_loan_columns['Column Name'] == feature_name]['Full Form'].squeeze()}\n"
            else:
                desc += f"Not {df_loan_columns[df_loan_columns['Column Name'] == feature_name]['Full Form'].squeeze()}\n"

            #desc += f"{df_loan_columns[df_loan_columns['Column Name'] == {feature_name}]['Full Form'].squeeze()}\n"
        print(desc)
    else:
        print("\nCongratulations! Your Loan Application is Approved. Following are the possible reasons we found from your Data\n")
        desc = ""
        for _, row in df_shap_features.iterrows():
            feature_name = row['feature']
            shap_value = row['shap_value']
            if(shap_value < 0):
                desc += f"Not {df_loan_columns[df_loan_columns['Column Name'] == feature_name]['Full Form'].squeeze()}\n"
            else:
                desc += f"{df_loan_columns[df_loan_columns['Column Name'] == feature_name]['Full Form'].squeeze()}\n"

            #desc += f"{df_loan_columns[df_loan_columns['Column Name'] == {feature_name}]['Full Form'].squeeze()}\n"
        print(desc)
generate_shap_descriptions(top_features.copy(), y_pred[0])

## Data Pre-processing Pipeline for Training purpose

In [ ]:
'''
# 1. Remove Non-Relevant Columns
def drop_nonrelevant_columns(dataframe : pd.DataFrame):
    # List of data leakage columns to REMOVE from LendingClub dataset
    leakage_columns = [
        # Payment history leakage
        'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv',
        'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee',
        'last_pymnt_amnt', 'last_pymnt_d', 'next_pymnt_d',

        # Recovery & collections leakage
        'recoveries', 'collection_recovery_fee', 'collections_12_mths_ex_med',

        # Investor-specific leakage
        'funded_amnt', 'funded_amnt_inv',

        # Date leakage (except issue_d which can be used carefully)
        'last_credit_pull_d', 'earliest_cr_line',  # Be careful with these

        # Any other columns containing these patterns
        'total_', 'out_', '_rec_', '_pymnt', 'last_', 'next_', 'recover'
    ]

    # Additional pattern-based removal (captures any missed leakage)
    leakage_patterns = ['total_', 'out_', '_rec_', '_pymnt', 'last_', 'next_', 'recover', 'collect', 'hardship_', 'deferral_']

    # Get all columns matching patterns
    pattern_leakage = [col for col in dataframe.columns
                   if any(pattern in col.lower() for pattern in leakage_patterns)]

    # Combine both lists (remove duplicates)
    all_leakage_columns = list(set(leakage_columns + pattern_leakage))

    # Keep only columns that actually exist in your dataset
    existing_leakage_columns = [col for col in all_leakage_columns if col in dataframe.columns]

    unnecessary_additional_columns = ['member_id', 'emp_title', 'issue_d', 'pymnt_plan', 'url', 'title', 'desc', 'zip_code', 'addr_state', 'payment_plan_start_date', 'debt_settlement_flag_date', 'settlement_date', 'initial_list_status', 'policy_code', 'verification_status_joint', 'annual_inc_joint', 'dti_joint', 'revol_bal_joint', 'orig_projected_additional_accrued_interest', 'settlement_status', 'settlement_amount', 'settlement_percentage', 'settlement_term', 'num_sats', 'chargeoff_within_12_mths', 'delinq_amnt', 'disbursement_method', 'grade', 'sub_grade', 'debt_settlement_flag', 'int_rate', 'installment']
    existing_leakage_columns.extend(unnecessary_additional_columns)
    #existing_leakage_columns

    dataframe_dropped = dataframe.copy()
    dataframe_dropped.drop(columns=existing_leakage_columns, inplace=True, errors='ignore')

    return dataframe_dropped

# 2. Drop rows where loan is running
def drop_running_loan(dataframe : pd.DataFrame):
    loan_status_current = dataframe[(dataframe['loan_status'] == 'Fully Paid') | (dataframe['loan_status'] == 'Charged Off') | (dataframe['loan_status'] == 'Default')].index

    #print(f"Index of Complete loan shappe = {len(loan_status_current)}")

    #df = dataframe.copy()
    #print(f"Before Drop Loan status = {dataframe['loan_status'].value_counts()}")
    #dataframe.drop(~loan_status_current, inplace=True)
    mask = dataframe.index.isin(loan_status_current)
    dataframe = dataframe[mask]
    #print(f"After Drop Loan status = {dataframe['loan_status'].value_counts()}")
    #print(f"If Any NA = {dataframe['loan_status'].isna().any()}")

    dataframe.loc[dataframe['loan_status'] == 'Fully Paid', 'loan_status'] = 1
    dataframe.loc[dataframe['loan_status'] == 'Charged Off', 'loan_status'] = 0
    dataframe.loc[dataframe['loan_status'] == 'Default', 'loan_status'] = 0

    #df.drop('loan_status', axis=1, inplace=True)
    return dataframe

# 3. Strip the values of Categorical features
def strip_categorical_values(dataframe : pd.DataFrame):
    categorical_columns = dataframe.select_dtypes(include='object_')
    for cc in categorical_columns:
        dataframe[cc] = dataframe[cc].apply(lambda x : str(x).strip())

    return dataframe

# 4. Replace rows with emp_length=nan with 'Unnown' since they might be doing business or wages where the length is not defined.
def replace_emp_length_nan(dataframe : pd.DataFrame):
    dataframe["emp_length"] = dataframe["emp_length"].where(dataframe["emp_length"] != "nan", "Unknown")

    return dataframe

# 5. Create OneHotEncoding of the Values where clear prioritization is not intuited
def onehotencode_columns(dataframe : pd.DataFrame):
    columns = ['term', 'home_ownership', 'purpose', 'verification_status']

    dataframe_temp = dataframe.copy()
    dataframe_temp.drop(columns, axis=1, inplace=True)

    df_onehotencoded_columns = pd.get_dummies(dataframe[columns], prefix=columns, dtype=np.int32)

    return pd.concat([dataframe_temp, df_onehotencoded_columns], join='outer', axis=1)

# 6. Manually Specify the Order of Ordinal Values
def fn_custom_orders(dataframe : pd.DataFrame):
    custom_orders = {
        'emp_length' : ['Unknown', '< 1 year', '1 year', '2 years', '3 years', '4 years', '5 years', '6 years', '7 years', '8 years', '9 years', '10+ years'],
        #'debt_settlement_flag' : ['N', 'Y'],
        'application_type' : ['Individual', 'Joint App']
    }
    for k,v in custom_orders.items():
        ordinalencoder = OrdinalEncoder(categories=[v])
        if(k in dataframe.columns):
            dataframe[k] = ordinalencoder.fit_transform(dataframe[[k]]).astype(np.int32)
    #print(f"Dataframe = {dataframe}")
    return dataframe

# 7. Fill NaN in the row values with their Median since the difference in the default rates and non-default rates is extremely low (See analyze_missingness)
def fill_nan_with_median(dataframe : pd.DataFrame):
    features = [
        'revol_util', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'inq_fi', 'bc_open_to_buy', 'bc_util', 'mo_sin_old_il_acct', 'mths_since_recent_bc', 'mths_since_recent_bc_dlq', 'mths_since_recent_inq', 'mths_since_recent_revol_delinq', 'num_tl_120dpd_2m', 'percent_bc_gt_75'
    ]
    df = dataframe.copy()
    for feature in features:
        #dataframe[feature].fillna(dataframe[feature].median(), inplace=True)
        df.fillna({feature : dataframe[feature].median()}, inplace=True)
    return df

# 8. Remove rows where dti is NaN and possibly annual_inc is 0
def remove_dti_nan(dataframe : pd.DataFrame):
    #dataframe.drop(dataframe[(dataframe["dti"].isna() == True) and (dataframe["annual_inc"].astype(np.int32) <= 0).index], inplace=True)
    df = dataframe.copy()
    df.drop(dataframe[(dataframe["dti"].isna() == True) & (dataframe["annual_inc"].astype(np.int32) <= 0)].index, inplace=True)
    return df

# 9. Create a Separate column for specifying if any secondary applicant is available
def sec_app_handle(dataframe : pd.DataFrame):
    MAX_FICO_SCORE = 900
    # 9.a Create a new feature to show if a secondary applicant is available
    sec_app_columns = [x for x in dataframe.columns if x.startswith("sec_app_")]
    dataframe['has_secondary_applicant'] = dataframe[sec_app_columns].notna().any(axis=1).astype(int)

    # 9.b Create a new feature for showing the credit-worthiness of the secondary applicant if present.
    # Following code gives Higher credit worth to individual applicants(without secondary applicant) which is not intuited.
    #dataframe['sec_app_fico_range_low'].fillna(0, axis=0, inplace=True)
    #dataframe['sec_app_revol_util'].fillna(0, axis=0, inplace=True)
    #dataframe["sec_app_worth"] = ((MAX_FICO_SCORE - dataframe["sec_app_fico_range_low"] / 100) + (dataframe['sec_app_revol_util'] / 100))

    dataframe['has_secondary_applicant'] = dataframe[['sec_app_fico_range_low', 'sec_app_revol_util']].notna().any(axis=1).astype(int)
    dataframe['sec_app_worth'] = 0
    #dataframe['sec_app_medium_risk'] = 0
    #dataframe['sec_app_high_risk'] = 0
    #print(f"Mid-point = {dataframe.head()}")
    has_sec_app = dataframe['has_secondary_applicant'] == 1

    dataframe.loc[has_sec_app & (dataframe['sec_app_fico_range_low'] >= 500) & (dataframe['sec_app_revol_util'] < 30), 'sec_app_worth'] = 3
    dataframe.loc[has_sec_app & (dataframe['sec_app_fico_range_low'] >= 500) & (dataframe['sec_app_revol_util'] >= 30), 'sec_app_worth'] = 2
    dataframe.loc[has_sec_app & (dataframe['sec_app_fico_range_low'] < 500) & (dataframe['sec_app_revol_util'] >= 30), 'sec_app_worth'] = 1
    dataframe.loc[has_sec_app & (dataframe['sec_app_fico_range_low'] < 500) & (dataframe['sec_app_revol_util'] < 30), 'sec_app_worth'] = 0

    # Drop the columns
    df = dataframe.copy()
    df.drop(sec_app_columns, axis=1, inplace=True)

    return df

# 10. Create a new feature based on the Average of fico_low and fico_high
def fico_average(dataframe : pd.DataFrame):
    dataframe["fico_score"] = (dataframe["fico_range_low"] + dataframe["fico_range_high"]) / 2

    dataframe.drop(columns=['fico_range_low', 'fico_range_high'], axis=1, inplace=True)
    return dataframe

# 11. Fill the columns with NaN with 0
def fill_na_0(dataframe : pd.DataFrame):
    df = dataframe.fillna(0).astype(np.int32)
    return df
'''

## Create Pipeline for the Dataframe Transformation

In [ ]:
'''
def data_preprocessing_pipeline(dataframe : pd.DataFrame):
    df_preprocessed = dataframe.pipe(drop_nonrelevant_columns).pipe(drop_running_loan).pipe(strip_categorical_values).pipe(replace_emp_length_nan).pipe(onehotencode_columns).pipe(fn_custom_orders).pipe(fill_nan_with_median).pipe(remove_dti_nan).pipe(sec_app_handle).pipe(fico_average).pipe(fill_na_0)

    #df_preprocessed.head()
    #print(f"Preprocessed Dataset\n{df_preprocessed}")
    return df_preprocessed
'''

## Use processed data

In [ ]:
'''
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df_tt = pd.DataFrame()
df_accepted = pd.DataFrame()
#df_preprocessed = []
for df_accepted_temp in pd.read_csv("/content/drive/MyDrive/Datasets/accepted.csv", chunksize=150000, index_col='id', low_memory=False):
  df_processed = df_accepted_temp.pipe(data_preprocessing_pipeline)
  df_accepted = pd.concat([df_accepted, df_processed], axis=0, ignore_index=True)
  df_accepted_temp = None
  #df_tt = df_accepted_temp
  #break

df_accepted = df_accepted.pipe(fill_na_0)
df_accepted.head()
#print("Data Converted Successfully")
#print(f"Dataframe shape = {df_accepted.shape}")
#df_tt.head()
'''